# QR Decomposition

QR decomposition is to decompose a matrix $A$ into a product $A=QR$ of an orthogonal matrix $Q$ and a upper triangular matrix $R$.

## Orthogonal Matrix

Let $Q = (q_1, q_2, \ldots, q_n)$ be an orthogonal matrix, $Q^T Q = QQ^T = I$

* For each vector $q_i$, $\langle q_i, q_i \rangle = 1$. i.e $\left|q_i\right| = 1$.
* For any two of the vectors $q_i$ and $q_j$, the inner product $\langle q_i, q_j \rangle = 0$, which means they are orthogonal.

$Q$ consists of $n$ orthogonal basis. Given a vector $v$ in original space, the coordinates in the new space defined by $Q$ is

$$u = Q^T v.$$

## Using householder reflections

To get a QR decomposition, we can construct householder reflections repeatedly. Given a matrix, as the first step, we want to find a orthogonal transformation $H_1$ such that only the first element in the first column is non-zero after the transformation.
$$
\begin{bmatrix}
x_{11} & x_{12} & \ldots & y_1\\
x_{21} & x_{22} & \ldots & y_2\\
\ldots & \ldots & \ldots & \ldots\\
x_{n1} & x_{n2} & \ldots & y_n\\
\end{bmatrix}
\xrightarrow{H_1}
\begin{bmatrix}
x_{11}^{*} & x_{12}^{*} & \ldots & y_1^{*}\\
0 & x_{22}^{*} & \ldots & y_2^{*}\\
\ldots & \ldots & \ldots & \ldots\\
0 & x_{n2}^{*} & \ldots & y_n^{*}\\
\end{bmatrix}
$$

Note that since the orthogonal transformation preserves the length of vectors, we know 
$$\left|x_1^*\right| = \left| x_1 \right| = \sqrt{(x_{11}^2 + x_{12}^2 + \ldots + x_{1n}^2)} ,$$
which means the value of $x_{11}$ is determined
$$x_{11}^* = \pm \left| x_1 \right|.$$

In actual the implementation, the sign is chosen as the opposite of $x_{11}$ for numerical stability.

To find a transformation $H$ which can rotate vector $x_1$ to $x_1^{*}$,
one simple way is to construct an isosceles triangle where $x_1^*$ is just a reflection of $x_1$.
$$x_1^* = x_1 - 2 \langle x_1, u \rangle u = x_1 - 2 u u^{T}x_1 = H_1 x_1,$$
where
$$u = \dfrac{x_1 \pm x_1^*}{ \left| x_1 \pm x_1^* \right| },\quad H_1 = I - 2 u u^{T}.$$

<img src="figures/householder-reflection.png">

To transform the original matrix $A_{n\times p}$ into a upper triganle matrix, we can repeat this procedure $p$ times, 
the orthogonal matrix $H = H_p\ldots H_2 H_1$.
Hence, $ HA = R$. let $Q = H^T$, we obtain the QR decomposition of $A$
$$
A  = QR.
$$

In [1]:
import numpy as np

def qr(A):
    n, m = A.shape
    R = A.copy()
    Q = np.eye(n)

    for k in range(m-1):
        x = np.zeros((n, 1))
        x[k:, 0] = R[k:, k]
        s = -1 * np.sign(x[k, 0])
        v = x
        v[k] = x[k] - s*np.linalg.norm(x)
        u = v / np.linalg.norm(v)
        
        R -= 2 * np.dot(u, np.dot(u.T, R))
        Q -= 2 * np.dot(u, np.dot(u.T, Q))
    return Q.T, R

In [2]:
# Test our QR function
A = np.array([[-2.0, 2, 3],
              [1, 3, 5],
              [-3, -1, 2]])

Q, R = qr(A)

print '[myqr] Q'
print Q.round(8)
print '[myqr] R'
print R.round(8)

Q_gt, R_gt = np.linalg.qr(A)
print '[numpy] Q'
print Q_gt
print '[numpy] R'
print R_gt

[myqr] Q
[[-0.53452248 -0.6172134  -0.57735027]
 [ 0.26726124 -0.77151675  0.57735027]
 [-0.80178373  0.15430335  0.57735027]]
[myqr] R
[[ 3.74165739  0.53452248 -1.87082869]
 [-0.         -3.7032804  -5.40061725]
 [ 0.          0.          2.30940108]]
[numpy] Q
[[-0.53452248 -0.6172134  -0.57735027]
 [ 0.26726124 -0.77151675  0.57735027]
 [-0.80178373  0.15430335  0.57735027]]
[numpy] R
[[ 3.74165739  0.53452248 -1.87082869]
 [ 0.         -3.7032804  -5.40061725]
 [ 0.          0.          2.30940108]]


## Solving least-squares using QR decompostion

Consider rotate the matrix $(X Y)$ by some orthogonal matrix $Q$,
$$\begin{bmatrix}X & Y\end{bmatrix} \xrightarrow{Q^T} \begin{bmatrix}R & Y^*\end{bmatrix} = \begin{bmatrix}R_1 & Y_1^*\\ 0 & Y_2^*\end{bmatrix},$$
where $R_1$ is a upper triangular matrix.

To solve the least square, 
$$ \min_{\beta} \left|Y^* - R\beta\right|^2 = \min_{\beta} \left(\left|Y_1^* -R_1\beta \right|^2 + \left| Y_2^* \right|^2\right).$$
So the solution $\hat{\beta} = R_1^{-1}Y_1^*$ and $\textrm{RSS} = \left|Y_2^*\right|^2$.

To solve $\hat{\beta}$ is essentially easy, since $R_1$ is an upper triangular matrix. We can solve the elements of $\hat{\beta}$ in reverse order $\hat{\beta}_p, \hat{\beta}_{p-1}, \ldots, \hat{\beta}_1$. It is numerically stable and efficient.

In [3]:
# Synthesis a dataset with n observations and p predictors.
n = 100
p = 5
X = np.random.random_sample((n, p))

# True coefficients are 1, 2, ..., p.
beta = np.array(range(1, p+1))
Y = np.dot(X, beta) + np.random.standard_normal(n)

# Stack (X Y) ans solve it by QR decomposition.
# Here we add the first column to be 1's for solving the intercepts.
Z = np.hstack((np.ones(n).reshape((n, 1)), X, Y.reshape((n, 1))))
_, R = qr(Z)
R1 = R[:p, :p]
Y1 = R[:p, p]

# Solve beta.
beta = np.linalg.solve(R1, Y1)
print beta

[ 0.54922972  0.10821076 -0.15395141 -0.07106197 -0.01305993]
